In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
batch_size=1, shuffle=True)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [4]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [5]:
def train(epoch):
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        # forward pass
        # here we could do different things on each pass...
        output = model(data)
        
        loss = F.nll_loss(output, target)
        # backward pass
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [6]:
def test():
    
    model.eval()
    
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),
100. * correct / len(test_loader.dataset)))

In [7]:
for epoch in range(1, 10):
    train(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.359755
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.631291
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.551744
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.529495
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.466105
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.335197
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.324826
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.295385
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.380579
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.251249
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.180389
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.180578
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.203153
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.204562
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.316000
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.274162
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.204789
Train Epoch: 3 [50000/60000 (83%)]	Loss: 0.303423
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.180434
Train Epoch: 4 [10000/60000 (17%)]	Loss: 0.168273
Train Epoch: 4 [2000

In [8]:
test()


Test set: Average loss: 0.0417, Accuracy: 9864/10000 (99%)

